In [5]:
!pip install transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
!pip install wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=69bb2b6f6faf4eb4bea8381f6638b010b122286b995fa215eb3a81ab1710602a
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


## Importing required libraries
It is recommended that you import all required libraries in one place (here):_


In [3]:
import wget
import torch
from transformers.models.dpr import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers.models.dpr import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import warnings

# suppress warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings('ignore')

In [34]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')
def read_and_split_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    # Split the text into paragraphs (simple split by newline characters)
    paragraphs = text.split('\n')
    # Filter out any empty paragraphs or undesired entries
    paragraphs = [para.strip() for para in paragraphs if len(para.strip()) > 0]
    return paragraphs

# Read the text file and split it into paragraphs
paragraphs = read_and_split_text('companyPolicies.txt')

policy = paragraphs[0:100]

file downloaded


In [5]:
DPR_context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
DPR_context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

DPRContextEncoder LOAD REPORT from: facebook/dpr-ctx_encoder-single-nq-base
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
ctx_encoder.bert_model.pooler.dense.bias   | UNEXPECTED |  | 
ctx_encoder.bert_model.pooler.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
DPR_question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
DPR_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

DPRQuestionEncoder LOAD REPORT from: facebook/dpr-question_encoder-single-nq-base
Key                                             | Status     |  | 
------------------------------------------------+------------+--+-
question_encoder.bert_model.pooler.dense.bias   | UNEXPECTED |  | 
question_encoder.bert_model.pooler.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [35]:
def token_encoder(text):
  embeddings = []
  text_token=DPR_context_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
  text_encoder = DPR_context_encoder(**text_token)
  embeddings.append(text_encoder.pooler_output)

  return torch.cat(embeddings).detach().numpy()

policy_encoder_oupt = token_encoder(policy)


In [37]:
# !pip install faiss-cpu
import faiss
def faiss_index(embedding,embedding_dim):
  dim = embedding_dim
  context_embeddings_np = np.array(embedding).astype('float32')
  index = faiss.IndexFlatIP(dim)
  faiss.normalize_L2(context_embeddings_np)
  index.add(context_embeddings_np)
  return index

faiss_index_output = faiss_index(policy_encoder_oupt,768)

In [11]:
GPT_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
GPT_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
GPT_model.generation_config.pad_token_id = GPT_tokenizer.pad_token_id

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: openai-community/gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [38]:
def question_encoder(question_text,libarary,libary_original_text,topn,):
  k = topn

  question_token=DPR_question_tokenizer(question_text, return_tensors='pt', padding=True, truncation=True, max_length=256)
  question_encoder = DPR_question_encoder(**question_token).pooler_output.detach().numpy()
  similarity_score,libarary_index = libarary.search(question_encoder, topn)
  topn_reteiveal_context=[libary_original_text[i] for i in libarary_index[0]]
  return topn_reteiveal_context


question = 'Drug and Alcohol Policy'
retrieval_result = question_encoder(question,faiss_index_output,policy,3)


In [ ]:
!pip install bert-score
from bert_score import score


In [39]:
import torch
import math

def calculate_perplexity(text):

    encodings = GPT_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    )

    input_ids = encodings["input_ids"]

    with torch.no_grad():
        outputs = GPT_model(
            input_ids=input_ids,
            labels=input_ids   # 关键：让模型计算语言模型loss
        )

    loss = outputs.loss
    perplexity = torch.exp(loss)

    return perplexity.item()


In [41]:
# def answer_generator(topn_reteiveal_context, max_len=50, min_len=40, length_penalty=2.0, num_beam=4):
def answer_generator(topn_reteiveal_context,max_len, min_len,length_penalty, num_beams):

  concated_text =' '.join(topn_reteiveal_context)

  GPT_tokenizer.pad_token = GPT_tokenizer.eos_token

  concated_text_token = GPT_tokenizer(
      concated_text,return_tensors='pt',
      max_length=1024,
      truncation=True,
      padding=True)

  generated_id = GPT_model.generate(
      concated_text_token['input_ids'],
      attention_mask=concated_text_token['attention_mask'],
      max_length=max_len,
      min_length = min_len,
      length_penalty =length_penalty,
      do_sample=num_beams,
      repetition_penalty = 1.2,
      no_repeat_ngram_size=3,
      early_stopping=True)

  input_ids = concated_text_token['input_ids']

  answer = GPT_tokenizer.decode(generated_id[0][input_ids.shape[1]:], skip_special_tokens=True)
  P, R, F1 = score([answer], [concated_text], lang="en", verbose=True)

  print("BERTScore Precision:", P.mean().item())
  print("BERTScore Recall:", R.mean().item())
  print("BERTScore F1:", F1.mean().item())



  perplexity = calculate_perplexity(answer)
  print("perplexity",perplexity)
  print("*******answer",answer)
  return answer


# answer_generator(retrieval_result, 50, 20 ,2.5, 5)


settings = [
    (40,10,0.8,4),
    (50, 50, 1.0, 2),
    (50, 20 ,2.5, 5)
    # (50, 30, 3.0, 6),
    # (50, 30, 3.5, 6)
]

for setting in settings:
    max_len, min_len, length_penalty, num_beams = setting   # 这里 _ 是 min_len，你的函数没用，所以可以忽略
    answer = answer_generator(retrieval_result, max_len=max_len, min_len = min_len,length_penalty=length_penalty, num_beams=num_beams)
    print(f"Settings: max_len={max_len}, length_penalty={length_penalty}, num_beams={num_beams}")
    print("Generated Answer:", answer)
    print("="*50)



The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.20 seconds, 0.84 sentences/sec
BERTScore Precision: 0.8260921239852905
BERTScore Recall: 0.8540059924125671
BERTScore F1: 0.8398171663284302
perplexity 638.5189208984375
*******answer  13.hovahLegalizing Abuse of Children 8,937 Criminal Law Reform 1820
Settings: max_len=50, length_penalty=2.5, num_beams=5
Generated Answer:  13.hovahLegalizing Abuse of Children 8,937 Criminal Law Reform 1820
